In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

from catboost import CatBoostRegressor
from catboost import Pool, CatBoostClassifier
import catboost

import xgboost as xgb
import prepro_util

In [2]:
### read test data
test = pd.read_csv('../../../Desktop/wids/test_data.csv')

In [3]:
### target column
target = 'contest-tmp2m-14d__tmp2m'

In [4]:
### test data
pre_test = prepro_util.preprocess_data(test , 4 , "mean" , "")

index and datetime set
location data handled
datetime handled


## CAT predict

In [19]:
cat_models = os.listdir('./CATboost/models/')
cat_models = [model for model in cat_models if '.json' in model]

In [20]:
result_df_list = []

In [21]:
for model in cat_models:
    temp_model = catboost.CatBoostClassifier()
    temp = temp_model.load_model(f"./CATboost/models/{model}")
    cat_predict_y = temp_model.predict(pre_test , prediction_type = 'RawFormulaVal' ,)

    result = test.copy()
    result[target] = cat_predict_y
    result = result.reset_index()
    result = pd.DataFrame(result[['index' , target]] , columns = ['index' , target])
    result_df_list.append(result)

In [22]:
for df in range(0 , len(result_df_list)):
    result_df_list[df].to_csv(f"./output/{cat_models[df]}.csv" , index = False)

## XGB predict

In [ ]:
xgb_model = xgb.Booster()
xgb_model.load_model("./models/XGB_mean_fill_max_depth_5.json")

In [ ]:
test_dataset = xgb.DMatrix(pre_test)
predict_y = xgb_model.predict(test_dataset)

In [ ]:
xgb_result = test.copy()
xgb_result[target] = predict_y
xgb_result = xgb_result.reset_index()

In [ ]:
xgb_result = pd.DataFrame(xgb_result[['index' , target]] , columns = ['index' , target])

In [ ]:
### create directory
try:
    os.makedirs("output")
except:
    pass

xgb_result.to_csv("./output/xgboost_mean_fill_maxD_5.csv" , index = False)